# Applied Data Science Capstone
## Week 5
### Opening a Café in Toronto

Import Libraries

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

Scrape Wikipedia

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text
soup = BeautifulSoup(data, 'html5lib')
table = soup.find('table')

Only include required data in the table and create a DataFrame 

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    # Clean the data i.e. ignore the neighbourhoods that are 'Not assigned'
    if row.span.text=='Not assigned':
        pass
    # Iclude 'Postal Code', 'Borough' and 'Neighborhood' data
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# Make DataFrame
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

Import geocoder and make function to obtain Longitudes and Latitudes for Toronto postcodes

In [4]:
import geocoder # import geocoder

def find_coords(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

Make a list of coordinates for all Toronto Post Codes

In [5]:
coord_list=[]
for i in df['PostalCode']:
    coord_list.append(find_coords(i))

Add coordinates into dataframe

In [6]:
df_coords = pd.DataFrame(coord_list, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Setup FourSquare API

In [8]:
address = 'Toronto, Ontario Canada'
geolocator = Nominatim(user_agent="trnt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto Canada are 43.6534817, -79.3839347.


#### Foursquare Credentials

In [9]:
CLIENT_ID = 'DZXOZM22TA3EHYBXMHPU1RFD1RQS2ZWBGVWG0U44OZQVMCGM' # your Foursquare ID
CLIENT_SECRET = 'GHAERUJF3ZYHOHFYYB3KBG5IFJMJTUVRPLVGU3RHRFCRMXJU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 500

## 2. Explore Neighborhoods in Toronto

#### Create function to get venue data for Toronto

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Create DataFrame of Toronto venues in Queen's Park and Downtown Toronto

In [11]:
toronto_data = df[df['Borough'].str.contains("Queen's Park|Downtown Toronto")].reset_index(drop=True)
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [12]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,63,63,63,63,63,63
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",80,80,80,80,80,80
Central Bay Street,76,76,76,76,76,76
Christie,10,10,10,10,10,10
Church and Wellesley,85,85,85,85,85,85
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Enclave of M5E,100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100


#### Create DataFrame for just cafés, coffee shops or tea rooms

In [13]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue'].unique())))

toronto_cafes=toronto_venues[(toronto_venues['Venue Category']=='Café') | (toronto_venues['Venue Category']=='Coffee Shop') | (toronto_venues['Venue Category']=='Tea Room') | (toronto_venues['Venue Category']=='Bubble Tea Shop')]
print('There are {} Cafés, Coffee Shops and Tea Rooms.'.format(len(toronto_cafes)))
toronto_cafes.head(15)

There are 719 uniques categories.
There are 201 Cafés, Coffee Shops and Tea Rooms.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
6,"Regent Park, Harbourfront",43.65512,-79.36264,Rooster Coffee,43.651900,-79.365609,Coffee Shop
8,"Regent Park, Harbourfront",43.65512,-79.36264,Sumach Espresso,43.658135,-79.359515,Coffee Shop
13,"Regent Park, Harbourfront",43.65512,-79.36264,Starbucks,43.651613,-79.364917,Coffee Shop
19,"Regent Park, Harbourfront",43.65512,-79.36264,Savoury Grounds,43.656821,-79.358970,Coffee Shop
29,Ontario Provincial Government,43.66253,-79.39188,Coffee Island,43.664271,-79.386972,Coffee Shop
32,Ontario Provincial Government,43.66253,-79.39188,Second Cup,43.659018,-79.394152,Coffee Shop
34,Ontario Provincial Government,43.66253,-79.39188,Aroma Espresso Bar,43.658928,-79.389903,Café
36,Ontario Provincial Government,43.66253,-79.39188,Tim Hortons,43.659415,-79.391221,Coffee Shop
37,Ontario Provincial Government,43.66253,-79.39188,Tim Hortons,43.658906,-79.388696,Coffee Shop


## 3. Create Heat Map of Cafés, Coffee Shops and Tea Rooms

In [15]:
from folium import plugins
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, name, neighborhood in zip(toronto_cafes['Venue Latitude'], toronto_cafes['Venue Longitude'], toronto_cafes['Venue'], toronto_cafes['Neighborhood']):
    label = '{}, {}'.format(name, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

hm_data = toronto_cafes[["Venue Latitude", "Venue Longitude"]]
map_toronto.add_child(plugins.HeatMap(hm_data))
map_toronto


#### Create DataFrame of Venues that are not Cafés, Coffee Shops and Tea Rooms

In [16]:
toronto_other=toronto_venues[(toronto_venues['Venue Category']!='Café') & (toronto_venues['Venue Category']!='Coffee Shop') & (toronto_venues['Venue Category']!='Tea Room') & (toronto_venues['Venue Category']!='Bubble Tea Shop')]
print('There are {} Other Venues.'.format(len(toronto_other)))
toronto_other

There are 1062 Other Venues.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.65512,-79.36264,Berkeley Church,43.655123,-79.365873,Event Space
5,"Regent Park, Harbourfront",43.65512,-79.36264,Body Blitz Spa East,43.654735,-79.359874,Spa
7,"Regent Park, Harbourfront",43.65512,-79.36264,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
9,"Regent Park, Harbourfront",43.65512,-79.36264,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
10,"Regent Park, Harbourfront",43.65512,-79.36264,Mangia and Bevi Resto-Bar,43.652250,-79.366355,Italian Restaurant
11,"Regent Park, Harbourfront",43.65512,-79.36264,Impact Kitchen,43.656369,-79.356980,Restaurant
12,"Regent Park, Harbourfront",43.65512,-79.36264,Sukhothai,43.658444,-79.365681,Thai Restaurant


#### Create map with other venues

In [17]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, name, neighborhood in zip(toronto_other['Venue Latitude'], toronto_other['Venue Longitude'], toronto_other['Venue'], toronto_other['Neighborhood']):
    label = '{}, {}'.format(name, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='None',
        fill=True,
        fill_color='black',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  

hm_data = toronto_cafes[["Venue Latitude", "Venue Longitude"]]
map_toronto.add_child(plugins.HeatMap(hm_data))
folium.Circle([43.65204,-79.38039], radius=200, color='white', fill=True, fill_opacity=0.6).add_to(map_toronto)
map_toronto

The black dots show all venues except Cafés, Coffee Shops and Tea Rooms, with the heat map for Cafés, Coffee Shops and Tea Rooms. The white circle has a radius of 200m and contains a large number of venues but no Cafés, Coffee Shops and Tea Rooms. Therefore, one would expect a large numbe of people to pass through this area without there being much competition for our new café. 